# EDA notebook

## Importing the Data

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import data_preparation as dp

bom_cleaned = pd.read_csv("../data/bom.movie_gross.csv.gz")
conn = sqlite3.connect("../data/im.db/im.db")

In [4]:
bom_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [12]:
for table_name in imdb_tables:
    df = pd.read_sql(f"SELECT * from {table_name[0]}", conn)
    print(f"Table: {table_name[0]}")
    print(df.info(),"\n")

Table: movie_basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
None 

Table: directors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB
None 

Table: known_for
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638260 entries, 0 to 16

## GENRE VS RATING

In [13]:
genre_vs_rating=pd.read_sql('''
SELECT b.genres, avg(r.averagerating) as average_rating
FROM movie_basics b
	JOIN movie_ratings r
	USING (movie_id)
GROUP BY genres;'''
, conn)
genre_vs_rating.head(20)
genre_vs_rating.iloc[:,:1].head()

,genres
0,None
1,Action
2,"Action,Adult,Comedy"
3,"Action,Adventure"
4,"Action,Adventure,Animation"


## GENRE VS GROSS

In [5]:
genre=pd.read_sql('''
SELECT primary_title as 'title', genres
FROM movie_basics
;'''
, conn)
pd.get_dummies(genre['genres'])
df[['First Name', 'Last Name']] = df.Student_details.str.split(",", expand = True)

,Action,"Action,Adult,Comedy","Action,Adventure","Action,Adventure,Animation","Action,Adventure,Biography","Action,Adventure,Comedy","Action,Adventure,Crime","Action,Adventure,Documentary","Action,Adventure,Drama","Action,Adventure,Family",...,"Sport,Talk-Show","Sport,Thriller",Talk-Show,Thriller,"Thriller,War","Thriller,War,Western","Thriller,Western",War,"War,Western",Western
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df = bom_cleaned.merge(genre,how='inner',on='title')

,title,studio,domestic_gross,foreign_gross,year,genres
0,Toy Story 3,BV,415000000.0,652000000,2010,"Adventure,Animation,Comedy"
1,Inception,WB,292600000.0,535700000,2010,"Action,Adventure,Sci-Fi"
2,Shrek Forever After,P/DW,238700000.0,513900000,2010,"Adventure,Animation,Comedy"
3,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010,"Adventure,Drama,Fantasy"
4,Iron Man 2,Par.,312400000.0,311500000,2010,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...
3361,Souvenir,Strand,11400.0,NaN,2018,"Comedy,Romance"
3362,Souvenir,Strand,11400.0,NaN,2018,Documentary
3363,Beauty and the Dogs,Osci.,8900.0,NaN,2018,"Crime,Drama,Thriller"
3364,The Quake,Magn.,6200.0,NaN,2018,"Action,Drama,Thriller"


## GENRE VS GROSS & BUDGET

## BUDGET VS GROSS

## WRITER VS GROSS

## DIRECTOR VS GROSS OR RATING

## RATED (R, M, PG, PG13) vs RATING (Popularity)

## (MAYBE TOO DEEP) DVD Sales vs Gross?	

## RUNTIME VS GROSS/RATING (LOOK BY GENRE)

## RATING(pg,13, R) VS GROSS	